In [3]:
import tweepy
import numpy as np
import pandas as pd
import json
import re

In [4]:
consumer_key = "YOUR KEY HERE"
consumer_secret = "YOUR KEY HERE"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

In [5]:
access_token = "YOUR TOKEN HERE"
access_token_secret = "YOUR TOKEN HERE"

auth.set_access_token(access_token, access_token_secret)

In [6]:
api = tweepy.API(auth)

In [11]:
before_vote_date_list = ['202102260000','202102270000','202102280000','202103010000','202103020000',
             '202103030000', '202103040000', '202103050000']
after_vote_date_list = ['202103060000', '202103070000', '202103080000', '202103090000', 
                       '202103100000', '202103110000', '202103120000', '202103130000']
## Last name for Senators with unique last names, full names for Senators whose last names 
## could have other meanings/uses
search_terms = ['Manchin', 'Sinema', 'Shaheen', 'Maggie Hassan', 'Carper', 'Jon Tester', 
                'Chris Coons', 'Angus King']
search_terms = [term + ' -"RT"' for term in search_terms]
senators = ["Joe Manchin", "Kyrsten Sinema", "Jeanne Shaheen", "Maggie Hassan", 
           "Tom Carper", "Jon Tester", "Chris Coons", "Angus King"]
frame_initiated = False

In [12]:
## Taken from StackOverflow response dealing with similar issue:
## https://stackoverflow.com/a/55562039

def get_tweet_text(tweet):
    try:
        return tweet.extended_tweet['full_text']
    except AttributeError as error:
        return tweet.text

In [13]:
for i, search_term in enumerate(search_terms):
    ## Collecting tweets in the before vote period
    for j in range((len(before_vote_date_list) - 1)):
        results = api.search_30_day('Development',
                                search_term,
                                fromDate = before_vote_date_list[j],
                                toDate = before_vote_date_list[j+1],
                                maxResults = 100)
        id_list = []
        date_list = []
        senator_list = []
        ## This binary variable will indicate whether the tweets came before or after the vote
        after_vote_list = []
        text_list = []

        for tweet in results:
            id_list.append(tweet.id_str)
            date_list.append(tweet.created_at)
            senator_list.append(senators[i])
            after_vote_list.append(0)
            text_list.append(get_tweet_text(tweet))

        InfoDict = {'ids': id_list, 'date': date_list, 'senator': senator_list,
                    'after_vote': after_vote_list, 'text': text_list}
        df_temp = pd.DataFrame.from_dict(InfoDict)
        if frame_initiated == False:
            senators_df = df_temp.copy()
            frame_initiated = True
        else:
            senators_df = pd.concat([senators_df, df_temp])
    
    ## Collecting tweets in the after vote period
    for j in range((len(after_vote_date_list) - 1)):
        results = api.search_30_day('Development',
                                search_term,
                                fromDate = after_vote_date_list[j],
                                toDate = after_vote_date_list[j+1],
                                maxResults = 100)
        id_list = []
        date_list = []
        senator_list = []
        after_vote_list = []
        text_list = []

        for tweet in results:
            id_list.append(tweet.id_str)
            date_list.append(tweet.created_at)
            senator_list.append(senators[i])
            after_vote_list.append(1)
            text_list.append(get_tweet_text(tweet))

        InfoDict = {'ids': id_list, 'date': date_list, 'senator': senator_list,
                    'after_vote': after_vote_list, 'text': text_list}
        df_temp = pd.DataFrame.from_dict(InfoDict)
        senators_df = pd.concat([senators_df, df_temp])

TweepError: {'message': 'Exceeded rate limit', 'sent': '2021-03-14T19:30:12+00:00', 'transactionId': '001ceb2800be5e94'}

In [14]:
senators_df.shape

(3000, 5)

In [15]:
senators_df.head()

,ids,date,senator,after_vote,text
0,1365451555076472836,2021-02-26 23:59:54,Joe Manchin,0,@f__of__x @angryprog @suchnerve Didn't Manchin...
1,1365451550261346305,2021-02-26 23:59:53,Joe Manchin,0,@JamaalBowmanNY Why are all the progressives s...
2,1365451486881325056,2021-02-26 23:59:37,Joe Manchin,0,@CAAttorney1 @JuddLegum You seem certain Manch...
3,1365451462831083523,2021-02-26 23:59:32,Joe Manchin,0,@MiddleFWIW @BlueCheese911 @cenkuygur @BernieS...
4,1365451442295775233,2021-02-26 23:59:27,Joe Manchin,0,@lorio808 @ACTBrigitte You could ask Steve man...


In [17]:
senators_df.senator.value_counts()

Joe Manchin       1400
Kyrsten Sinema    1400
Jeanne Shaheen     200
Name: senator, dtype: int64

In [27]:
## Came across rate limit, will have to try again with remaining Senators. In meanwhile, will 
## filter to just two that I have so far
full_senators_df = senators_df.loc[(senators_df.senator == "Joe Manchin") | (senators_df.senator == "Kyrsten Sinema")]
del senators_df
full_senators_df.shape

(2800, 5)

In [28]:
full_senators_df.to_csv("senator_tweets.csv")

In [30]:
def scrape_senator_tweets(search_term_list, given_senator_list, df): 
    senators_df = df
    for i, search_term in enumerate(search_term_list):
        ## Collecting tweets in the before vote period
        for j in range((len(before_vote_date_list) - 1)):
            results = api.search_30_day('Development',
                                    search_term,
                                    fromDate = before_vote_date_list[j],
                                    toDate = before_vote_date_list[j+1],
                                    maxResults = 100)
            id_list = []
            date_list = []
            senator_list = []
            ## This binary variable will indicate whether the tweets came before or after the vote
            after_vote_list = []
            text_list = []

            for tweet in results:
                id_list.append(tweet.id_str)
                date_list.append(tweet.created_at)
                senator_list.append(given_senator_list[i])
                after_vote_list.append(0)
                text_list.append(get_tweet_text(tweet))

            InfoDict = {'ids': id_list, 'date': date_list, 'senator': senator_list,
                        'after_vote': after_vote_list, 'text': text_list}
            df_temp = pd.DataFrame.from_dict(InfoDict)
            senators_df = pd.concat([senators_df, df_temp])

        ## Collecting tweets in the after vote period
        for j in range((len(after_vote_date_list) - 1)):
            results = api.search_30_day('Development',
                                    search_term,
                                    fromDate = after_vote_date_list[j],
                                    toDate = after_vote_date_list[j+1],
                                    maxResults = 100)
            id_list = []
            date_list = []
            senator_list = []
            after_vote_list = []
            text_list = []

            for tweet in results:
                id_list.append(tweet.id_str)
                date_list.append(tweet.created_at)
                senator_list.append(given_senator_list[i])
                after_vote_list.append(1)
                text_list.append(get_tweet_text(tweet))

            InfoDict = {'ids': id_list, 'date': date_list, 'senator': senator_list,
                        'after_vote': after_vote_list, 'text': text_list}
            df_temp = pd.DataFrame.from_dict(InfoDict)
            senators_df = pd.concat([senators_df, df_temp])
    return senators_df

In [31]:
part_search_terms = search_terms[2:4]
part_senators = senators[2:4]

In [32]:
full_senators_df = scrape_senator_tweets(part_search_terms, part_senators, full_senators_df)

In [33]:
full_senators_df

,ids,date,senator,after_vote,text
0,1365451555076472836,2021-02-26 23:59:54,Joe Manchin,0,@f__of__x @angryprog @suchnerve Didn't Manchin...
1,1365451550261346305,2021-02-26 23:59:53,Joe Manchin,0,@JamaalBowmanNY Why are all the progressives s...
2,1365451486881325056,2021-02-26 23:59:37,Joe Manchin,0,@CAAttorney1 @JuddLegum You seem certain Manch...
3,1365451462831083523,2021-02-26 23:59:32,Joe Manchin,0,@MiddleFWIW @BlueCheese911 @cenkuygur @BernieS...
4,1365451442295775233,2021-02-26 23:59:27,Joe Manchin,0,@lorio808 @ACTBrigitte You could ask Steve man...
...,...,...,...,...,...
68,1370177944178585601,2021-03-12 01:00:53,Maggie Hassan,1,@Tony_Demers @Tomato8816 @WMUR9 @SaintAnselmPo...
69,1370176807140462592,2021-03-12 00:56:22,Maggie Hassan,1,@Tony_Demers @Tomato8816 @WMUR9 @SaintAnselmPo...
70,1370176729080332289,2021-03-12 00:56:03,Maggie Hassan,1,@Tony_Demers @Tomato8816 @WMUR9 @SaintAnselmPo...
71,1370167125151727622,2021-03-12 00:17:53,Maggie Hassan,1,@pt35mm @Tomato8816 @WMUR9 @SaintAnselmPoll @C...


In [34]:
full_senators_df.senator.value_counts()

Joe Manchin       1400
Jeanne Shaheen    1400
Kyrsten Sinema    1400
Maggie Hassan     1002
Name: senator, dtype: int64

In [35]:
full_senators_df.to_csv("senator_tweets.csv")

In [36]:
part_search_terms = search_terms[4:6]
part_senators = senators[4:6]
full_senators_df = scrape_senator_tweets(part_search_terms, part_senators, full_senators_df)

In [38]:
full_senators_df.senator.value_counts()

Joe Manchin       1400
Jeanne Shaheen    1400
Kyrsten Sinema    1400
Tom Carper        1090
Maggie Hassan     1002
Jon Tester         785
Name: senator, dtype: int64

In [39]:
full_senators_df.to_csv("senator_tweets.csv")

In [40]:
part_search_terms = search_terms[6:]
part_senators = senators[6:]
full_senators_df = scrape_senator_tweets(part_search_terms, part_senators, full_senators_df)

In [41]:
full_senators_df.senator.value_counts()

Joe Manchin       1400
Jeanne Shaheen    1400
Kyrsten Sinema    1400
Tom Carper        1090
Maggie Hassan     1002
Angus King         854
Chris Coons        802
Jon Tester         785
Name: senator, dtype: int64

In [42]:
full_senators_df.to_csv("senator_tweets.csv")